<a href="https://colab.research.google.com/github/ettoreotery/MSA/blob/main/AMD_Ettore_Oteri_v2_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "ettoreoteri"
os.environ['KAGGLE_KEY'] = "xxxxxxxxxxxxxxxxxxxxxxxxxxx"
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

!unzip -q "*.zip" -d /content/

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0


In [ ]:
import pandas as pd
df1 = pd.read_csv("Books_rating.csv")
df1.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [ ]:
print(f"This dataset contains {df1.shape[0]} rows and {df1.shape[1]} columns.")

This dataset contains 3000000 rows and 10 columns.


In [ ]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from itertools import combinations

# Download the required NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('punkt_tab', quiet=True)

True

In [ ]:
# Tokenization function
def safe_tokenize(text):
    try:
        return word_tokenize(text)
    except:
        return text.split()  # Simple fallback

# Text preprocessing setup
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
translator = str.maketrans('', '', string.punctuation)

def preprocess_text(text):
    if not isinstance(text, str):
        return []

    # Text normalization
    text = text.lower().translate(translator)
    tokens = safe_tokenize(text)
    # Return sorted tuple of unique lemmatized tokens
    return tuple(sorted(set(lemmatizer.lemmatize(word) for word in tokens
                      if word not in stop_words and len(word) > 2)))

# Data loading and cleaning
print("Loading and cleaning data...")
df = pd.read_csv("Books_rating.csv")

# Remove exact duplicates
df = df.drop_duplicates(subset=['review/text'], keep='first')

# Sample 1,000 reviews and preprocess
df = df.sample(1000, random_state=36)
df['tokens'] = df['review/text'].apply(preprocess_text)

# Remove duplicates after preprocessing
df = df.drop_duplicates(subset=['tokens'], keep='first')

# Filter reviews with at least 5 unique tokens
df = df[df['tokens'].apply(len) >= 5]
reviews = df['tokens'].tolist()

Loading and cleaning data...


In [ ]:
# Similarity calculation with duplicate checks
print("Calculating similarities...")

top_pairs = []
checked_pairs = set()

for i, j in combinations(range(len(reviews)), 2):
    if reviews[i] == reviews[j]:  # Skip identical reviews
        continue

    pair_key = frozenset({i,j})
    if pair_key in checked_pairs:  # Avoid duplicate comparisons
        continue

    checked_pairs.add(pair_key)
    set1 = set(reviews[i])
    set2 = set(reviews[j])
    sim = len(set1 & set2) / len(set1 | set2) if (set1 | set2) else 0
    top_pairs.append((sim, i, j))

# Sort and select top 20 pairs
top_pairs.sort(reverse=True, key=lambda x: x[0])
top_20 = top_pairs[:20]

# Display results
print("\nTOP 20 MOST SIMILAR PAIRS:")
for rank, (sim, i, j) in enumerate(top_20, 1):
    orig_text_i = df.iloc[i]['review/text'][:100] + ('...' if len(df.iloc[i]['review/text']) > 100 else '')
    orig_text_j = df.iloc[j]['review/text'][:100] + ('...' if len(df.iloc[j]['review/text']) > 100 else '')

    print(f"\n#{rank}: Similarity = {sim:.4f}")
    print(f"Review {i}: {orig_text_i}")
    print(f"Review {j}: {orig_text_j}")
    print(f"Common tokens: {set(reviews[i]) & set(reviews[j])}")

# Save results
results_df = pd.DataFrame(top_20, columns=['similarity', 'index1', 'index2'])
results_df['text1'] = results_df['index1'].apply(lambda x: df.iloc[x]['review/text'])
results_df['text2'] = results_df['index2'].apply(lambda x: df.iloc[x]['review/text'])
results_df.to_csv('top_pairs_no_duplicates.csv', index=False)

Calculating similarities...

TOP 20 MOST SIMILAR PAIRS:

#1: Similarity = 0.2857
Review 759: If you are into books about Jesus and his life then you must get this. I have read it over and over ...
Review 933: I have read the book and realy enjoyed it. I also have the DVD of it. You realy must get both.
Common tokens: {'book', 'get', 'read', 'must'}

#2: Similarity = 0.2727
Review 118: This was an incredible book I can't wait for the seaquel
Review 916: I have yet to read the book, but I have heard good things about it. Can't wait to start it!
Common tokens: {'book', 'wait', 'cant'}

#3: Similarity = 0.2632
Review 168: This book is on the Kumon required reading list. Wonderful illustrations, amusing, and easy to read....
Review 425: It's not hype, it truly is a great read. Wonderful story, easy reading. Great book to just kick back...
Common tokens: {'book', 'wonderful', 'reading', 'read', 'easy'}

#4: Similarity = 0.2500
Review 75: This is an awesome book about grieving for all ages. I